<a href="https://colab.research.google.com/github/SvA1/Compling/blob/master/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter, defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from scipy import spatial

In [0]:
!wget http://vectors.nlpl.eu/repository/20/180.zip

In [0]:
import nltk
nltk.download('stopwords')
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [0]:
def normalize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]
    return ' '.join(words)

In [0]:
def tokenize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    return ' '.join(words)

**Моя модель**

In [0]:
data = open('corpus_hum.txt').read().splitlines()

In [0]:
data_norm = [normalize(text) for text in tqdm(data)]

In [0]:
data_norm = [text for text in tqdm(data_norm) if text]

In [0]:
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=60, sg=1)

In [0]:
w2v.save('w2v.model')

**Модель с rusvectores** 


In [0]:
#from zipfile import ZipFile
#with ZipFile("180.zip", "r") as zipObj:
#  zipObj.extractall()

In [0]:
w2v_vectores = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

**Перифразы**

In [0]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []

classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [0]:
data['text_1_norm'] = tqdm(data['text_1'].apply(normalize))

In [0]:
data['text_2_norm'] = tqdm(data['text_2'].apply(normalize))

In [0]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

**Embeddings**

In [0]:
def embedding(text, model, dim):
    text = text.split()
    
    
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) 
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [0]:
dim = 60
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = embedding(text, w2v, dim)

In [0]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [0]:
y = data['label'].values

In [0]:
clf = LogisticRegression(C=1000, class_weight='balanced')

In [0]:
scores_1 = cross_val_score(clf, X_text_w2v, y, cv=5, scoring='f1_macro')

In [89]:
scores_1.mean()

0.389981025763696

In [0]:
data2= pd.read_csv('data_paraphraser_norm.csv')

In [0]:
dim = 300
X2_text_1_w2v = np.zeros((len(data2['text_1_norm']), dim))
X2_text_2_w2v = np.zeros((len(data2['text_2_norm']), dim))

for i, text in enumerate(data2['text_1_norm'].values):
    X2_text_1_w2v[i] = embedding(text, w2v_vectores, dim)
    
for i, text in enumerate(data2['text_2_norm'].values):
    X2_text_2_w2v[i] = embedding(text, w2v_vectores, dim)

In [0]:
X2_text_w2v = np.concatenate([X2_text_1_w2v, X2_text_2_w2v], axis=1)

In [0]:
scores_2 = cross_val_score(clf, X2_text_w2v, y, cv=5, scoring='f1_macro')

In [90]:
scores_2.mean()

0.38830880522151473

**Вывод**

Моя модель и модель с русвекторес показали почти одинаковый результат

## Задание 2

**SVD**

In [0]:
svd = TruncatedSVD(800, n_iter=20, random_state=20)

In [0]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

In [112]:
svd.fit(tfidf.transform(pd.concat([data['text_1_norm'], data['text_2_norm']])))

TruncatedSVD(algorithm='randomized', n_components=800, n_iter=20,
             random_state=20, tol=0.0)

In [0]:
X_text_1_svd = svd.transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd = svd.transform(tfidf.transform(data['text_2_norm']))

X_text_svd = np.concatenate([X_text_1_svd, X_text_2_svd], axis=1)

In [0]:
scores_svd = cross_val_score(clf, X_text_svd, y, cv=5, scoring='f1_macro')

In [120]:
scores_svd.mean()

0.37646884473504405

**Cos SVD**

In [0]:
distances_svd = np.zeros(data2.shape[0])

In [0]:
for i in range(len(data['text_1_norm'])):
    distances_svd[i] = cosine_distances(X_text_1_svd[[i]], X_text_2_svd[[i]])[0][0]

In [0]:
data2['cos_svd'] = distances_svd

**NMF**

In [0]:
nmf = NMF(70, init='nndsvd', random_state=20, alpha=0.4, tol=1e-1, solver='mu')
nmf.fit(tfidf.transform(pd.concat([data['text_1_norm'], data['text_2_norm']])))

In [0]:
X_text_1_nmf = nmf.transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.transform(tfidf.transform(data['text_2_norm']))

X_text_nmf = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

In [0]:
scores_nmf = cross_val_score(clf, X_text_nmf, y, cv=5, scoring='f1_macro')

In [133]:
scores_nmf.mean()

0.370844973493279

**Cos NMF**

In [0]:
distances_nmf = np.zeros(data2.shape[0])

In [0]:
for i in range(len(data2['text_1_norm'])):
    distances_nmf[i]  = cosine_distances(X_text_1_nmf[[i]], X_text_2_nmf[[i]])[0][0]

In [0]:
data2['cos_nmf'] = distances_nmf

**Моя W2V модель**

In [0]:
dim = 60
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = embedding(text, w2v, dim)

**Cos моей W2V**

In [0]:
distances_w2v = np.zeros(data2.shape[0])

In [0]:
for i in range(len(data2['text_1_norm'])):
    distances_w2v[i]  = cosine_distances(X_text_1_w2v[[i]], X_text_2_w2v[[i]])[0][0]

In [0]:
data2['cos_w2v'] = distances_w2v

**W2V с rusvectores**

In [0]:
dim = 300
X2_text_1_w2v = np.zeros((len(data2['text_1_norm']), dim))
X2_text_2_w2v = np.zeros((len(data2['text_2_norm']), dim))

for i, text in enumerate(data2['text_1_norm'].values):
    X2_text_1_w2v[i] = embedding(text, w2v_vectores, dim)
    
for i, text in enumerate(data2['text_2_norm'].values):
    X2_text_2_w2v[i] = embedding(text, w2v_vectores, dim)

**Cos W2V rusvectores**

In [0]:
distances_w2v_vectores = np.zeros(data2.shape[0])

In [0]:
for i in range(len(data2['text_1_norm'])):
    distances_w2v_vectores[i]  = cosine_distances(X2_text_1_w2v[[i]], X2_text_2_w2v[[i]])[0][0]

In [0]:
data2['cos_w2v_vectores'] = distances_w2v_vectores

**Fastext**

In [0]:
fast_text = gensim.models.FastText([text.split() for text in data_norm], size=80, min_n=3, max_n=6)

In [0]:
dim = 80
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = embedding(text, fast_text, dim)

In [0]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

In [0]:
scores_ft = cross_val_score(clf, X_text_ft, y, cv=5, scoring='f1_macro')

In [189]:
scores_ft

array([0.40764852, 0.43682611, 0.44199221, 0.39234539, 0.34110069])

**Cos fasttext**

In [0]:
distances_ft = np.zeros(data2.shape[0])

In [0]:
for i in range(len(data2['text_1_norm'])):
    distances_ft[i]  = cosine_distances(X_text_1_ft[[i]], X_text_2_ft[[i]])[0][0]

In [0]:
data2['cos_ ft'] = distances_ft

In [193]:
data2.head(4)

,label,text_1,text_2,text_1_norm,text_2_norm,cos_w2v,cos_svd,cos_w2v_vectores,cos_nmf,cos_ ft,text_1_norm_no_pos,text_2_norm_no_pos
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...,0.105070,0.645362,0.315568,0.958973,0.178968,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...,0.081876,0.559198,0.245257,0.005366,0.247972,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...,0.054577,0.299560,0.173560,0.199697,0.359533,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться_VERB из_ADP сирия_NOUN россиянин_NOU...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...,0.224668,0.613035,0.678593,0.169838,0.366406,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия


**Cosine distance**

In [0]:
data2['text_1_norm_no_pos'] = tqdm(data2['text_1'].apply(normalize))

In [0]:
data2['text_2_norm_no_pos'] = tqdm(data2['text_2'].apply(normalize))

**Кроссвалидация**

In [0]:
X_cos = data2.iloc[:,6:10].values

In [0]:
scores_5 = cross_val_score(clf, X_cos, y, cv=5, scoring='f1_macro')

In [198]:
scores_5

array([0.50955254, 0.54568245, 0.56943436, 0.48004768, 0.50143625])